# Learning to Rank на основе LETOR MQ2008
## Анализ данных, построение и оценка ранжирующей модели




In [32]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("taoqin/letor4")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'letor4' dataset.
Path to dataset files: /kaggle/input/letor4


In [48]:
import os
import pandas as pd
from lightgbm import LGBMModel, Dataset, train

In [34]:
fold_path = "/kaggle/input/letor4/MQ2008/Fold1"


print("Проверка наличия файлов:")
print("train.txt:", os.path.isfile(os.path.join(fold_path, "train.txt")))
print("vali.txt:", os.path.isfile(os.path.join(fold_path, "vali.txt")))
print("test.txt:", os.path.isfile(os.path.join(fold_path, "test.txt")))

Проверка наличия файлов:
train.txt: True
vali.txt: True
test.txt: True


In [35]:
def load_letor_data(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split('#')[0].split()
            label = int(parts[0])
            qid = int(parts[1].split(':')[1])
            features = {f'feature_{i}': 0.0 for i in range(1, 47)}

            for elem in parts[2:]:
                if ':' in elem:
                    fid, value = elem.split(':')
                    fid = int(fid)
                    if 1 <= fid <= 46:
                        features[f'feature_{fid}'] = float(value)
                    else:
                      print(f"Недопуститмый признак. elem: {elem}, fid: {fid}" )

            row = {'label': label, 'qid': qid}
            row.update(features)
            data.append(row)

    df = pd.DataFrame(data)
    return df

In [36]:
train_df = load_letor_data(os.path.join(fold_path, "train.txt"))
print(f"Тренировочный сет: {train_df.head(5)}")
print(f"Размер: {train_df.shape}")

vali_df = load_letor_data(os.path.join(fold_path, "vali.txt"))
print(f"Валидационный сет: {vali_df.head(5)}")
print(f"Размер: {vali_df.shape}")

test_df = load_letor_data(os.path.join(fold_path, "test.txt"))
print(f"Тестовый сет: {test_df.head(5)}")
print(f"Размер: {test_df.shape}")

Тренировочный сет:    label    qid  feature_1  feature_2  feature_3  feature_4  feature_5  \
0      0  10002   0.007477        0.0        1.0        0.0   0.007470   
1      0  10002   0.603738        0.0        1.0        0.0   0.603175   
2      0  10002   0.214953        0.0        0.0        0.0   0.213819   
3      0  10002   0.000000        0.0        1.0        0.0   0.000000   
4      0  10002   1.000000        1.0        0.0        0.0   1.000000   

   feature_6  feature_7  feature_8  ...  feature_37  feature_38  feature_39  \
0        0.0        0.0        0.0  ...    0.797056    0.697327    0.721953   
1        0.0        0.0        0.0  ...    0.000000    0.000000    0.117399   
2        0.0        0.0        0.0  ...    0.566409    0.760916    0.746370   
3        0.0        0.0        0.0  ...    0.320586    0.133604    0.000000   
4        0.0        0.0        0.0  ...    0.341228    0.292567    0.385569   

   feature_40  feature_41  feature_42  feature_43  feature_44

##EDA


In [37]:
train_df['label'].value_counts()

,count
label,
0,7820
1,1223
2,587


In [38]:
vali_df['label'].value_counts()

,count
label,
0,2140
1,400
2,167


In [39]:
test_df['label'].value_counts()

,count
label,
0,2319
1,378
2,177


Наблюдается дисбаланс классов - класс 0 преобладает над остальными. Логично для специфики задачи - для конкретного запроса во всем интернете найдется больше нерелевантных документов, чем релевантных

In [40]:
train_label_stats = train_df.groupby('qid')['label'].agg(['max']).value_counts()
train_label_stats

,count
max,
2,200
1,139
0,132


In [41]:
vali_label_stats = vali_df.groupby('qid')['label'].agg(['size', 'max']).describe()
vali_label_stats

,size,max
count,157.000000,157.000000
mean,17.242038,1.197452
std,17.278414,0.796140
min,6.000000,0.000000
25%,8.000000,1.000000
50%,8.000000,1.000000
75%,16.000000,2.000000
max,118.000000,2.000000


In [42]:
test_label_stats = test_df.groupby('qid')['label'].agg(['max']).value_counts()
test_label_stats

,count
max,
2,63
0,51
1,42


Можно увидеть, что для некотрых запросов отсутсвуют релевантные документы и в таком случае, задача ранжирования для них неактуальна, однако не стоит удалять эти запросы из наших данных так, как в релаьном мире пользователи также могут оставлять некорректные запросы и наша цель - сформировать хоть какую-то выдачу. Также если мы удалим такие запросы, то поднимем итоговые метрики, однако они будут неактуальны потому, что, как я и сказал раньше, при использовании могут приходить "плохие данные", с которыми му не будем знать, что делать.


Также по анализу валидационной выборки можно увидеть разброс в количестве документов для разных запросов (от 6 до 118). Это может негативно сказаться на обучении, ведь тогда модель будет уделять внимание наибольшим группам и не уловить зависимости для маленьких. Для минимизации этой проблемы предлагается мониторить качество не только в целом, но и по подвыборкам разного размера.


#Обучение

In [43]:
group_train = train_df.sort_values(by="qid").groupby("qid").size().tolist()
group_vali = vali_df.sort_values(by="qid").groupby("qid").size().tolist()
group_test = test_df.sort_values(by="qid").groupby("qid").size().tolist()

In [45]:
x_train = train_df.drop(['label', 'qid'], axis=1)
y_train = train_df['label']
x_vali = vali_df.drop(['label', 'qid'], axis=1)
y_vali = vali_df['label']

In [49]:
train_data = Dataset(x_train, label=y_train, group=group_train)
vali_data = Dataset(x_vali, label=y_vali, group=group_vali, reference=train_data)

In [59]:
params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'eval_at': [5, 10],
    'learning_rate': 0.05,
    'verbose': 1,
    'num_iterations': 500,
    'early_stopping_rounds': 50
}

model = train(params,
                  train_data,
                  valid_sets=[vali_data],
                  valid_names=['vali'])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002788 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9233
[LightGBM] [Info] Number of data points in the train set: 9630, number of used features: 40
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[16]	vali's ndcg@5: 0.747068	vali's ndcg@10: 0.793359
